In [92]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("max_columns", None)
%matplotlib inline

In [93]:
CAdata = pd.read_csv('CAdata.csv',low_memory=False,encoding = "ISO-8859-1")

In [94]:
#replace 
CAdata.columns = [x.replace(' ', '_') for x in CAdata.columns]

### Grouping Amenities using Logical OR operator on dummy columns of Amenities
### e.g. When either Air conditioning or Heading is True, Amenity Group 'temerature_regulation' is True

In [95]:
CAdata['parking'] =CAdata['Free_Parking_on_Premises'] | CAdata['Free_Parking_on_Street']| CAdata['Paid_Parking_Off_Premises']

In [96]:
CAdata['temerature_regulation'] = CAdata['Heating'] | CAdata['Air_Conditioning']

In [97]:
CAdata['tv']=CAdata['TV'] | CAdata['Cable_TV']

In [98]:
#Washer/Dryer
CAdata['in_unit_laundry']=CAdata['Washer'] | CAdata['Cable_TV']

In [99]:
CAdata['internet_access']=CAdata['Internet'] | CAdata['Wireless_Internet']

In [100]:
#Gym/Pool = recreation
CAdata['recreation']=CAdata['Gym'] | CAdata['Pool']

## List of amenity_groups we created

In [101]:
amenity_groups = ['parking','temerature_regulation','tv','in_unit_laundry','internet_access','recreation']

In [102]:
# Free parking on street/Free Parking on Premises/Paid Parking Off Premises
# Heating / Air Conditioning
# TV / Cable TV
# Washer/Dryer
# Breakfast
# Kitchen
# Internet/Wireless internet
# Gym/Pool
# Suitable for Events
# Elevator in Building

## Amenities/ amenity_groups we are considering for analysing for property type price infliuencers

In [103]:
proptype_amenities_list = ['parking','temerature_regulation','tv','in_unit_laundry','Breakfast','Kitchen','internet_access','recreation','Suitable_for_Events','Elevator_in_Building']

In [104]:
categorical_for_proptype =['host_is_superhost',
'neighbourhood_cleansed',
'room_type',
'security_deposit',
'cleaning_fee',
'extra_people',
'review_scores_rating',
'cancellation_policy']

In [105]:

proptype_numeric_features =['Host_Since',
'host_listings_count',
'accommodates',
'bathrooms',
'bedrooms',
'beds',
'number_of_reviews',
'reviews_per_month']

In [106]:
##Recode to binary categorical  0 when na, 1 when value is present
CAdata['security_deposit'] = CAdata['security_deposit'].map(lambda x: 0 if np.isnan(x) else 1 )


In [107]:
CAdata['cleaning_fee'] = CAdata['cleaning_fee'].map(lambda x: 0 if np.isnan(x) else 1 )

In [108]:
CAdata['extra_people'] = CAdata['extra_people'].map(lambda x: 0 if np.isnan(x) or x==0 else 1 )

In [109]:
CAdata['review_scores_rating',].describe()

KeyError: ('review_scores_rating',)

In [ ]:
def categorize_review_scores_rating(x):
    ret = 'review_rating_na'
    if x<89:
        ret='low'
    elif x>=89 and x<=98.25:
        # 25th and 75h percentile range
        ret = 'medium'
    elif x>98.25:
        ret ='high'
    else:
        ret ='review_rating_na'
    return ret



In [ ]:
CAdata['review_scores_rating'] = CAdata['review_scores_rating'].map(categorize_review_scores_rating )

In [ ]:
def select_property_types(x):
    ret = 'not_selected_for_analysis'
    if x =='Apartment':
        ret = 'Apartment'
    elif x == 'House' or x =='Townhouse' or x== 'Villa':
        ret = 'Independent_House'
    elif x =='Condominium':
        ret = 'Condominium'
    return ret

    

In [118]:
CAdata['property_type'] = CAdata['property_type'].map(select_property_types)

In [119]:
CAdata.property_type.value_counts()

Apartment                    2612
Independent_House             622
Condominium                   231
not_selected_for_analysis     120
Name: property_type, dtype: int64

In [ ]:
proptype = CAdata[['price','property_type'] + proptype_numeric_features+proptype_amenities_list+categorical_for_proptype]

In [ ]:
prop_type_with_dummies_dropped_first = pd.get_dummies(proptype,columns=categorical_for_proptype,drop_first=True)

In [112]:
proptype_Apartment = prop_type_with_dummies_dropped_first[prop_type_with_dummies_dropped_first['property_type']=='Apartment'].drop('property_type',axis =1)

In [120]:
proptype_Independent_House = prop_type_with_dummies_dropped_first[prop_type_with_dummies_dropped_first['property_type']=='Independent_House'].drop('property_type',axis =1)

In [121]:
proptype_Condominium = prop_type_with_dummies_dropped_first[prop_type_with_dummies_dropped_first['property_type']=='Condominium'].drop('property_type',axis =1)

In [122]:
len(proptype_Condominium)

231

In [131]:
proptype_Apartment.to_csv('proptype_Apartment.csv',index=False)

In [132]:
proptype_Independent_House.to_csv('proptype_Independent_House.csv',index=False)

In [133]:
proptype_Condominium.to_csv('proptype_Condominium.csv',index=False)